## Convertion of h5ad to loom & Tokenization
#### This is always the first step of feeding a h5ad file into a Geneformer-based model, no matter it is to fine-tune the foundational model or to apply prediction/extract embeddings.

#### Geneformer tokenizer requires ENSG IDs instead of gene symbols. This notebook uses the GProfiler package to match ENSG IDs with gene symbols automatically, which is a online searching process and may be prohibited/interrupted when running on server. Therefore, this notebook is preferably running locally. 

#### The equivalent python version of this notebook is h5ad_to_loom.py, where ENSG IDs are supposed to be ready in the h5ad file and online searching is no longer needed. Therefore, h5ad_to_loom.py is preferably running on server, especially when the h5ad file is too large to load locally.

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import scanpy as sc
from scipy.io import mmread
import torch
import loompy
from gprofiler import GProfiler
from geneformer import TranscriptomeTokenizer


os.environ['PYTHONHASHSEED'] = '0'
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
# Load genes, barcodes, and matrix if h5ad is unavailable, otherwise load h5ad directly 
name = 'mouse'
ref_directory = 'mouse/'

with open(ref_directory + 'genes.tsv') as f:
    genes = f.read().rstrip().split('\n')

with open(ref_directory + 'barcodes.tsv') as f:
    barcodes = f.read().rstrip().split('\n')

mat = mmread(ref_directory + 'matrix.mtx')
df = pd.DataFrame.sparse.from_spmatrix(mat, index=genes, columns=barcodes).fillna(0)
adata = sc.AnnData(df.T)
del mat, df
adata

In [ ]:
name = 'cao_cerebellum_developing_brain'
ref_directory = 'cao_cerebellum_developing_brain/'

adata = sc.read_h5ad(ref_directory + f'{name}.h5ad')
adata

In [ ]:
adata = adata[880000:, :].copy()
adata

In [ ]:
adata.obs

In [ ]:
adata.obs.index = adata.obs['CellID'].values
adata.obs

In [ ]:
adata.var

In [ ]:
# Ensure raw counts instead of log
np.amax(adata.X.toarray())

In [ ]:
np.amin(adata.X.toarray())

In [ ]:
# Load and attach cell metadata
index_col = 'CellID'
df_ref_meta = pd.read_csv(ref_directory + 'meta.tsv', sep='\t', index_col=index_col)
df_ref_meta = df_ref_meta.loc[adata.obs_names, :]
adata.obs = df_ref_meta.copy()
adata.obs

In [ ]:
# Load and filter cells in ephys dataset
df_ephys = pd.read_csv(ref_directory + 'human_mouse_ephys_all_0127.csv', index_col='specimen_id')
df_ephys = df_ephys.dropna(how='all') # drop cells whose ephys features are all none
df_ephys

In [ ]:
# Filter adata cells
adata = adata[adata.obs['SpecimenID'].isin(df_ephys.index.tolist())].copy()
adata

In [ ]:
# # Filter adata cells
# c1 = adata.obs['Input.resistance.(MOhm)'].notna()
# c2 = adata.obs['AP.amplitude.(mV)'].notna()
# c3 = adata.obs['Max.number.of.APs'].notna()
# adata = adata[c1 & c2 & c3].copy()
# adata

In [ ]:
# Save filtered adata
adata.write(ref_directory + f'{name}.h5ad')

In [ ]:
# Save filtered ephys dataset
df_ephys = df_ephys.loc[adata.obs['SpecimenID'].tolist(), :]
df_ephys.to_csv(ref_directory + 'human_mouse_ephys_all_0127_sorted283humanOnly.csv')
df_ephys

In [ ]:
# adata.obs['group'] = adata.obs['isImmune'].values # Designate one column as "group" that contains groups information for model finetuning
adata.obs['group'] = '_'
adata.obs['isTumor'] = 0 # A trick related to finetuning: only those cells labeled with "isTumor = 0" are to be used for model finetuning
adata.obs = adata.obs[['group', 'isTumor']].copy()

In [ ]:
# # Balance dataset among groups by subsampling, only performed when adata is to be used for model fine-tuning
# cells = []
# celltype_cells = {}
# n_cells = 1000
# for celltype, group in adata.obs.groupby('group'):
#     cells_this_type = group.index.tolist()
#     np.random.shuffle(cells_this_type)
#     cells = cells + cells_this_type[:n_cells]
#     celltype_cells[celltype] = cells_this_type[:n_cells]

# np.random.shuffle(cells)
# adata = adata[cells, :].copy()
# adata

### Online searching for ENSG IDs

In [ ]:
# Search and get initial results
gp = GProfiler(return_dataframe=True)
df_genes_converted = gp.convert(organism='hsapiens',
                                query=adata.var_names.tolist(),
                                target_namespace='ENSG')
df_genes_converted

In [ ]:
# Search and get initial results
gp = GProfiler(return_dataframe=True)
df_genes_converted = gp.orth(organism='mmusculus', query=adata.var_names.tolist(), target='hsapiens')
df_genes_converted

In [ ]:
# One gene symbol can match multiple or none ENSG IDs
# Drop duplicates and remove nones 
df_genes_converted = df_genes_converted[~df_genes_converted['incoming'].duplicated()]
df_genes_converted = df_genes_converted[~df_genes_converted['converted'].isin([None, np.nan, 'None', 'N/A'])]
# df_genes_converted = df_genes_converted[~df_genes_converted['ortholog_ensg'].isin([None, np.nan, 'None', 'N/A'])]
df_genes_converted

In [ ]:
# Save ENSG IDs, can be skipped
df_genes_converted[['incoming', 'converted', 'name', 'description']].to_excel(ref_directory + f'{name}_convertedGenes.xlsx', index=False)
# df_genes_converted[['incoming', 'converted', 'ortholog_ensg', 'name', 'description']].to_excel(ref_directory + f'{name}_convertedGenes.xlsx', index=False)

In [ ]:
# Filter out those genes with no ENSG IDs
adata = adata[:, df_genes_converted['incoming'].tolist()].copy()
adata

In [ ]:
# Add metadata required by tokenizer, don't change the feature names
adata.var['ensembl_id'] = df_genes_converted['converted'].tolist()
adata.obs['n_counts'] = np.sum(adata.X.toarray(), axis=1) # total read counts in each cell
adata.obs['filter_pass'] = 1
adata.obs['individual'] = adata.obs.index.tolist() # cell IDs

In [ ]:
# # Add metadata required by tokenizer (the case ENSG IDs are provided in the h5ad file)
# adata.var['ensembl_id'] = adata.var.index.tolist()
# adata.obs['n_counts'] = np.sum(adata.X.toarray(), axis=1) # total read counts in each cell
# adata.obs['filter_pass'] = 1
# adata.obs['individual'] = adata.obs['SpecimenID'].tolist() # cell IDs

### Saving as loom & Tokenization

In [ ]:
# Save as [name].loom in [ref_directory]
data = adata.X.toarray().T
df_row_metadata = adata.var.copy()
df_col_metadata = adata.obs.copy()
loompy.create(f'{ref_directory}{name}.loom', data, df_row_metadata.to_dict('list'), df_col_metadata.to_dict('list'))
del adata, data

In [ ]:
# Tokenize [name].loom
# Ensure [name].loom is the only loom file in [ref_directory]
# Output is a folder [name].dataset in [ref_directory]
tk = TranscriptomeTokenizer({'individual': 'individual', 'isTumor': 'isTumor', 'group': 'group', 'n_counts': 'n_counts'}, nproc=1)
tk.tokenize_data(ref_directory, ref_directory, name)
os.remove(f'{ref_directory}{name}.loom')